In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-earthquake-dataset-stead/merge.hdf5
/kaggle/input/stanford-earthquake-dataset-stead/merge.csv


Para trabajar con el dataset, utilizamos los servicios de kaggle que nos permiten guardar el input de manera persistente, lo cual es conveniente al ser un dataset de ~90GB.

In [2]:
import os
import pandas as pd

input_dir = '/kaggle/input/stanford-earthquake-dataset-stead/'
filename = 'merge.csv'
df_original = pd.read_csv(os.path.join(input_dir, filename), low_memory=False)
df_original.head()

,network_code,receiver_code,receiver_type,receiver_latitude,receiver_longitude,receiver_elevation_m,p_arrival_sample,p_status,p_weight,p_travel_sec,...,source_magnitude_author,source_mechanism_strike_dip_rake,source_distance_deg,source_distance_km,back_azimuth_deg,snr_db,coda_end_sample,trace_start_time,trace_category,trace_name
0,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-21 05:55:00,noise,109C.TA_201510210555_NO
1,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-06 14:50:00,noise,109C.TA_201511061450_NO
2,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-07 02:20:00,noise,109C.TA_201511070220_NO
3,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-11-14 05:15:00,noise,109C.TA_201511140515_NO
4,TA,109C,HH,32.8889,-117.1051,150.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-12-25 18:50:00,noise,109C.TA_201512251850_NO


Realizamos una copia del dataset, para ejecutar desde este punto las celdas, y no tener que volver a leer el dataset a pandas

In [ ]:
df = df_original.copy()
df.info()

Se trata de un dataset con 1265657 filas y 35 columnas.

La variable trace_name es un identificador único para cada observación:

In [ ]:
len(df['trace_name'].unique()) == len(df)

Por tanto, lo podemos utilizar como índice de fila.

In [ ]:
df = df.set_index('trace_name')
df.head()

network_code, receiver_code, trace_start_time y trace_category se especifican en el trace_name  

trace_name = [receiver_code].[network_code]\_[trace_start_time]\_[trace_category]

Observaciones:
- trace_start_time se expresa como unión sin espacios de las unidades de tiempo
- trace_category toma valores NO y EV para categorías noise y earthquake_local respectivamente

Se convierten a formato fecha las variables source_origin_time y trace_start_time

In [ ]:
df['trace_start_time'] = pd.to_datetime(df['trace_start_time'], format='ISO8601')# format="%Y-%m-%d %H:%M:%S")
df['source_origin_time'] = pd.to_datetime(df['source_origin_time'], format='ISO8601')

In [ ]:
df.iloc[0]['trace_start_time']

Se han recogido dos tipos de señales:  
- noise: sonidos no debidos a terremotos
- earthquake_local: terremotos que se encuentran en un radio menor a 350km

In [ ]:
df.loc[:, 'trace_category'].unique()

Para estudios que vamos a realizar, vamos dividir en dataframes uno por cada tipo de señal:

### Señales noise

Obtenemos un subdataset cuyas observaciones son señales noise, y podemos ver el ratio y el tamaño del subconjunto

In [ ]:
df_noise = df.loc[df.loc[:, 'trace_category'] == 'noise']
size = len(df_noise)
print(f'len(df_noise)={size}, ratio={size/len(df)*100:.2f}%')
df_noise.head()

En la documentación del dataset se muestra que las señales categorizadas como noise, solo cuentan con 8 variables y el resto toman valores NA, eliminamos las variables que tienen todos sus valores NA:

In [ ]:
df_noise = df_noise.dropna(axis=1, how='all')
print(df_noise.shape)
df_noise.head()

Una vez hecho esto, podemos comprobar si existen más valores NA, debidos a incosistencias.

In [ ]:
print(f'Número total de NA en el dataframe: {(col_na:=df_noise.isna().sum()).sum()}')
df_noise_na = pd.DataFrame({'Valores NA': col_na})
df_noise_na

Realizamos comprobaciones similares para los datos de terremotos.

In [ ]:
df_earthquakes = df.loc[df.loc[:, 'trace_category'] == 'earthquake_local']
size = len(df_earthquakes)
print(f'len(df_earthquakes)={size}, ratio={size/len(df)*100:.2f}%')
df_earthquakes.head()

In [ ]:
print(f'Número total de NA en el dataframe: {(col_na:=df_earthquakes.isna().sum()).sum()}')
df_earthquakes_na = pd.DataFrame({'Valores NA': col_na})
df_earthquakes_na.loc[df_earthquakes_na.loc[:, 'Valores NA'] != 0, :] # Mostramos solo los que no tienen 0

In [ ]:
df_earthquakes = df_earthquakes.dropna(axis=1, subset=['network_code', 'p_weight', 's_weight', 'source_depth_km'])

In [ ]:
import matplotlib.pyplot as plt
na_data = df_earthquakes_na.loc[df_earthquakes_na.loc[:, 'Valores NA'] != 0, :]
plt.figure(figsize=(10, 6))
plt.hist(na_data, bins=df_earthquakes_na.columns, alpha=0.7, rwidth=0.85)
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.title('Histograma de los valores de una única columna')
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# reemplazar NaN con la media de la columna correspondiente
#imputer = SimpleImputer(strategy='mean')

# Preparación de los datos
X = df_earthquakes[['source_latitude', 'source_longitude', 'source_depth_km']]  # predictores
y = df_earthquakes['source_magnitude']  # var objetivo

X.isna().sum()

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# reemplazar NaN con la media de la columna correspondiente
#imputer = SimpleImputer(strategy='mean')

# Preparación de los datos
X = df_earthquakes[['source_latitude', 'source_longitude', 'source_depth_km']]  # predictores
y = df_earthquakes['source_magnitude']  # var objetivo

# Imputar valores NaN
# X = imputer.fit_transform(X)

#datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir el conjunto de prueba
y_pred = model.predict(X_test)

# las métricas de rendimiento
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'R^2: {r2}')

Podemos comprobar mediante source_id como se distribuyen el número de muestras que hacen referencia al mismo terremoto

In [ ]:
earthquake_samples = df_earthquakes.groupby('source_id')
sizes = earthquake_samples.size()
sizes.describe()

In [ ]:
df_earthquakes.groupby('source_magnitude_type').size().plot(kind='bar')

In [ ]:
df_earthquakes.groupby('source_magnitude').size().plot(kind='bar')